In [1]:
import sys
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import json, base64
from time import sleep

In [2]:
def send_devtools(driver, cmd, params={}):
    resource = "/session/%s/chromium/send_command_and_get_result" % driver.session_id
    url = driver.command_executor._url + resource
    body = json.dumps({'cmd': cmd, 'params': params})
    response = driver.command_executor._request('POST', url, body)
    return response.get('value')

def get_pdf_from_html(path, chromedriver='/usr/local/bin/chromedriver', print_options = {}):
    options = webdriver.ChromeOptions()
    options.add_argument('--ignore-certificate-errors')
    options.add_argument("--test-type")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--headless")
    options.add_argument('--disable-gpu')
    driver = webdriver.Chrome(chromedriver, options=options)
    driver.get(path)
    sleep(3)
    calculated_print_options = {
        'landscape': False,
        'displayHeaderFooter': False,
        'printBackground': True,
        'preferCSSPageSize': True,
    }
    calculated_print_options.update(print_options)
    result = send_devtools(driver, "Page.printToPDF", calculated_print_options)
    driver.quit()
    return base64.b64decode(result['data'])

In [3]:
result = get_pdf_from_html('http://unicef.localhost/word')
with open('../public/download/results.pdf', 'wb') as file:
    file.write(result)

In [4]:
import docx

In [ ]:
docx.Document